In [ ]:
!pip install simpy

In [ ]:
import matplotlib.pyplot as plt
import simpy
import numpy as np
from sympy import *

# For A

In [ ]:
#our model with conccurent arrival in two diffrent queues
class A:
    def __init__(self, env, lam, mu_1, mu_2, lam_u):
        self.env = env
        self.q_1 = simpy.Store(env)
        self.q_2 = simpy.Store(env)
        self.mean_update = 1/(lam_u)
        self.mean_arrival = 2/lam
        self.mean_service_1 = 1/mu_1
        self.mean_service_2 = 1/mu_2
        # Arrival(queue's number, customer's calss)     Departure(queue's number, customer's calss)
        self.data = {'Arrival(Q1,C1)':[], 'Departure(Q1,C1)':[], 'Arrival(Q2,C2)':[], 'Departure(Q2,C1)':[], 'Departure(Q2,C2)':[]}

    def queue_1(self):
        while True:
            x =  yield self.q_1.get()
            if x == 'C1':
                yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_1)))
                self.data['Departure(Q1,C1)'].append(np.longfloat(self.env.now))
                self.q_2.put('C1')
            elif x == 'u':
                yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_1)))


    def queue_2(self):
        while True:
            c = yield self.q_2.get()
            if c == 'C1':
                yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_2)))
                self.data['Departure(Q2,C1)'].append(np.longfloat(self.env.now))
            elif c == 'C2':
                yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_2)))
                self.data['Departure(Q2,C2)'].append(np.longfloat(self.env.now))

    def arrival_1(self):
        while True:
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_arrival)))
            self.data['Arrival(Q1,C1)'].append(np.longfloat(self.env.now))
            self.q_1.put('C1')
            self.data['Arrival(Q1,C1)'].append(np.longfloat(self.env.now))
            self.q_1.put('C1')


    def arrival_2(self):
        while True:
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_arrival)))
            self.data['Arrival(Q2,C2)'].append(np.longfloat(self.env.now))
            self.q_2.put('C2')
            self.data['Arrival(Q2,C2)'].append(np.longfloat(self.env.now))
            self.q_2.put('C2')


    def update_file(self):
        while True:
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_update)))
            self.q_1.put('u')
            self.q_1.put('u')


Test the accuracy of the simulation.

In [ ]:
lam, mu_1, mu_2, lam_u = 3, 5, 10, 0.25 # request rate, service 1 rate, service 2 rate, update rate

env = simpy.Environment()
net = A(env, lam, mu_1, mu_2, lam_u)
env.process(net.arrival_1())
env.process(net.arrival_2())
env.process(net.queue_2())
env.process(net.queue_1())
env.process(net.update_file())
env.run(2*10**6)

In [ ]:
l_1 = len(net.data['Departure(Q2,C1)'])
l_2 = len(net.data['Departure(Q2,C2)'])
l1 = int(l_1*0.3)   #margin
l2 = int(l_2*0.3)   #margin

w_1a = (sum(net.data['Departure(Q2,C1)'][l1:-l1+l_1]) - sum(net.data['Arrival(Q1,C1)'][l1:-l1+l_1])) / len(net.data['Departure(Q2,C1)'][l1:-l1+l_1])
w_2a = (sum(net.data['Departure(Q2,C2)'][l2:-l2+l_2]) - sum(net.data['Arrival(Q2,C2)'][l2:-l2+l_2])) / len(net.data['Departure(Q2,C2)'][l2:-l2+l_2])
print(w_1a, w_2a)
sim = (w_1a + w_2a)/2

#theory of above simulation
p1a = (3 * lam + 6 * lam_u) / (2 * mu_1 + lam + 2 * lam_u)
p2a = (5 * lam + lam * (mu_1)/(mu_1 + mu_2)) / (2 * mu_2 + lam + lam * (mu_1)/(mu_1 + mu_2))

w_1a = (p1a / (1 - p1a)) * (1 / (lam + 2 * lam_u))
w_2a = (p2a / (1 - p2a)) * (1 / (2 * lam))

theo = (2*w_1a + 4*w_2a)/4

print(theo)
print("Error :", (sim - theo)*100/sim)

# For B

In [ ]:
#our model with conccurent arrival in two diffrent queues
class B:
    def __init__(self, env, lam, mu_1, mu_2, lam_u):
        self.env = env
        self.q_1 = simpy.Store(env)
        self.q_2 = simpy.Store(env)
        self.mean_update = 1/(2*lam_u)
        self.mean_arrival = 1/lam
        self.mean_service_1 = 1/mu_1
        self.mean_service_2 = 1/mu_2
        # Arrival(queue's number, customer's calss)     Departure(queue's number, customer's calss)
        self.data = {'Arrival(Q1,C1)':[], 'Departure(Q1,C1)':[], 'Arrival(Q2,C2)':[], 'Departure(Q2,C1)':[], 'Departure(Q2,C2)':[]}

    def queue_1(self):
        while True:
            x =  yield self.q_1.get()
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_1)))
            if x == 'C1':
                self.data['Arrival(Q2,C2)'].append(np.longfloat(self.env.now))
                self.q_2.put('C2')

                self.data['Departure(Q1,C1)'].append(np.longfloat(self.env.now))
                self.q_2.put('C1')


    def queue_2(self):
        while True:
            c = yield self.q_2.get()
            if c == 'C1':
                yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_2)))
                self.data['Departure(Q2,C1)'].append(np.longfloat(self.env.now))
            elif c == 'C2':
                yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_2)))
                self.data['Departure(Q2,C2)'].append(np.longfloat(self.env.now))

    def arrival(self):
        while True:
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_arrival)))
            self.data['Arrival(Q1,C1)'].append(np.longfloat(self.env.now))
            self.q_1.put('C1')



    def update_file(self):
        while True:
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_update)))
            self.q_1.put('u')

In [ ]:
lam, mu_1, mu_2, lam_u = 1, 5, 10, 0.25 # request rate, service 1 rate, service 2 rate, update rate

env = simpy.Environment()
net = B(env, lam, mu_1, mu_2, lam_u)
env.process(net.arrival())
env.process(net.queue_2())
env.process(net.queue_1())
env.process(net.update_file())
env.run(2*10**6)

In [ ]:
l_1 = len(net.data['Departure(Q2,C1)'])
l_2 = len(net.data['Departure(Q2,C2)'])
l1 = int(l_1*0.3)   #margin
l2 = int(l_2*0.3)   #margin

w_1b = (sum(net.data['Departure(Q2,C1)'][l1:-l1+l_1]) - sum(net.data['Arrival(Q1,C1)'][l1:-l1+l_1])) / len(net.data['Departure(Q2,C1)'][l1:-l1+l_1])
w_2b = (sum(net.data['Departure(Q2,C2)'][l2:-l2+l_2]) - sum(net.data['Arrival(Q2,C2)'][l2:-l2+l_2])) / len(net.data['Departure(Q2,C2)'][l2:-l2+l_2])
print(w_1b, w_2b)
sim = (w_1b + w_2b)/2

#theory of above simulation
p2b = (3 * lam) / (mu_2 + lam)

w_1b = 1 / (mu_1 - lam - 2 * lam_u)
w_2b = (p2b / (1 - p2b)) * (1 / (2 * lam))

theo = (w_1b / 2) + w_2b

print(theo)
print("Error :", (sim - theo)*100/sim)

# For C

In [ ]:
#our model with conccurent arrival in two diffrent queues
class C:
    def __init__(self, env, lam, mu_1, mu_2, lam_u):
        self.env = env
        self.q_1 = simpy.Store(env)
        self.q_2 = simpy.Store(env)
        self.mean_update = 1/(lam_u*2)
        self.mean_arrival = 1/lam
        self.mean_service_1 = 1/mu_1
        self.mean_service_2 = 1/mu_2
        # Arrival(queue's number, customer's calss)     Departure(queue's number, customer's calss)
        self.data = {'Arrival(Q1,C1)':[], 'Departure(Q1,C1)':[], 'Arrival(Q2,C2)':[], 'Departure(Q2,C1)':[], 'Departure(Q2,C2)':[]}

    def queue_1(self):
        while True:
            x =  yield self.q_1.get()
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_1)))
            if x == 'C1':
                self.data['Departure(Q1,C1)'].append(np.longfloat(self.env.now))
                self.q_2.put('C1')

    def queue_2(self):
        while True:
            c = yield self.q_2.get()
            if c == 'C1':
                yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_2)))
                self.data['Departure(Q2,C1)'].append(np.longfloat(self.env.now))
            elif c == 'C2':
                yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_service_2)))
                self.data['Departure(Q2,C2)'].append(np.longfloat(self.env.now))

    def arrival(self):
        while True:
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_arrival)))
            self.data['Arrival(Q1,C1)'].append(np.longfloat(self.env.now))
            self.q_1.put('C1')
            self.data['Arrival(Q2,C2)'].append(np.longfloat(self.env.now))
            self.q_2.put('C2')


    def update_file(self):
        while True:
            yield self.env.timeout(np.longfloat(np.random.exponential(self.mean_update)))
            self.q_1.put('u')

In [ ]:
lam, mu_1, mu_2, lam_u = 3, 5, 10, 0.5  # request rate, service 1 rate, service 2 rate, update rate

env = simpy.Environment()
net = C(env, lam, mu_1, mu_2, lam_u)
env.process(net.arrival())
env.process(net.queue_2())
env.process(net.queue_1())
env.process(net.update_file())
env.run(10**6)

In [ ]:
l_1 = len(net.data['Departure(Q2,C1)'])
l_2 = len(net.data['Departure(Q2,C2)'])
l1 = int(l_1*0.3)   #margin
l2 = int(l_2*0.3)   #margin

w_1c = (sum(net.data['Departure(Q2,C1)'][l1:-l1+l_1]) - sum(net.data['Arrival(Q1,C1)'][l1:-l1+l_1])) / len(net.data['Departure(Q2,C1)'][l1:-l1+l_1])
w_2c = (sum(net.data['Departure(Q2,C2)'][l2:-l2+l_2]) - sum(net.data['Arrival(Q2,C2)'][l2:-l2+l_2])) / len(net.data['Departure(Q2,C2)'][l2:-l2+l_2])
print(w_1c, w_2c)
sim = (w_1c + w_2c)/2

#theory of above simulation
p2c = (2 * lam + lam * (mu_1 / (mu_1 + mu_2))) / (mu_2 + lam * (mu_1 / (mu_1 + mu_2)))
p1c = (lam + lam_u)/(mu_1)



w_1c = (p1c / (1 - p1c)) * (1 / (lam + lam_u))
w_2c = (p2c / (1 - p2c)) * (1 / (2 * lam))

theo = (w_1c / 2) + w_2c


print(theo)
print("Error :", (sim - theo)*100/sim)

# Evalution for the variation of lambda for A

In [ ]:
import pandas as pd

data_A = {'Lambda':[], 'Simulation':[], 'Theory':[], 'Error':[]}

n = 40
for i in np.linspace(1, 4.5, 5):
    lam, mu_1, mu_2, lam_u = i, 30, 10, 0.5   # request rate, service 1 rate, service 2 rate, update rate

    data_A['Lambda'].append(lam)
    ############################################################################
    #simulation A
    env = simpy.Environment()
    net = A(env, lam, mu_1, mu_2, lam_u)
    env.process(net.arrival_1())
    env.process(net.arrival_2())
    env.process(net.queue_2())
    env.process(net.queue_1())
    env.process(net.update_file())
    env.run(n*10**5)
    n = n-5

    l_1 = len(net.data['Departure(Q2,C1)'])
    l_2 = len(net.data['Departure(Q2,C2)'])
    l1 = int(l_1*0.35)   #margin
    l2 = int(l_2*0.35)   #margin

    w_1a = (sum(net.data['Departure(Q2,C1)'][l1:-l1+l_1]) - sum(net.data['Arrival(Q1,C1)'][l1:-l1+l_1])) / len(net.data['Departure(Q2,C1)'][l1:-l1+l_1])
    w_2a = (sum(net.data['Departure(Q2,C2)'][l2:-l2+l_2]) - sum(net.data['Arrival(Q2,C2)'][l2:-l2+l_2])) / len(net.data['Departure(Q2,C2)'][l2:-l2+l_2])
    sim_a = (w_1a + w_2a)/2
    data_A['Simulation'].append(round(sim_a, 3))


    #theory of above simulation
    p1a = (3 * lam + 6 * lam_u) / (2 * mu_1 + lam + 2 * lam_u)
    p2a = (5 * lam + lam * (mu_1)/(mu_1 + mu_2)) / (2 * mu_2 + lam + lam * (mu_1)/(mu_1 + mu_2))

    w_1a = (p1a / (1 - p1a)) * (1 / (lam + 2 * lam_u))
    w_2a = (p2a / (1 - p2a)) * (1 / (2 * lam))

    theo_a = (2*w_1a + 4*w_2a)/4

    data_A['Theory'].append(round(theo_a, 3))
    data_A['Error'].append(round((sim_a - theo_a)*100/sim_a, 3))


In [ ]:
data_A

In [ ]:
x1 = [1.0, 1.625, 2.25, 2.875, 3.5]
ya1 = [0.417, 0.514, 0.67, 0.978, 1.948]  #mu_1 = 5, mu_2 = 10, lam_u = 0.5


x2 = [1.0, 1.875, 2.75, 3.625, 4.5]          #mu_1 = 30, mu_2 = 10, lam_u = 0.5
ya2 = [0.207, 0.259, 0.350, 0.560, 1.481]

x3 = [1.0, 1.75, 2.5, 3.25, 4.0]
ya3 = [0.267, 0.319, 0.397, 0.536, 0.87]    #mu_1 = 10, mu_2 = 10, lam_u = 0.5

# Evalution for the variation of lambda for B

In [ ]:
data_B = {'Lambda':[], 'Simulation':[], 'Theory':[], 'Error':[]}
n = 40

for i in np.linspace(1, 4.5, 5):
    lam, mu_1, mu_2, lam_u = i, 30, 10, 0.5  # request rate, service 1 rate, service 2 rate, update rate

    data_B['Lambda'].append(lam)

    ############################################################################
    #simulation B
    env = simpy.Environment()
    net = B(env, lam, mu_1, mu_2, lam_u)
    env.process(net.arrival())
    env.process(net.queue_2())
    env.process(net.queue_1())
    env.process(net.update_file())
    env.run(n*10**5)
    n = n-5

    l_1 = len(net.data['Departure(Q2,C1)'])
    l_2 = len(net.data['Departure(Q2,C2)'])
    l1 = int(l_1*0.3)   #margin
    l2 = int(l_2*0.3)   #margin

    w_1b = (sum(net.data['Departure(Q2,C1)'][l1:-l1+l_1]) - sum(net.data['Arrival(Q1,C1)'][l1:-l1+l_1])) / len(net.data['Departure(Q2,C1)'][l1:-l1+l_1])
    w_2b = (sum(net.data['Departure(Q2,C2)'][l2:-l2+l_2]) - sum(net.data['Arrival(Q2,C2)'][l2:-l2+l_2])) / len(net.data['Departure(Q2,C2)'][l2:-l2+l_2])
    sim_b = (w_1b + w_2b)/2
    data_B['Simulation'].append(round(sim_b, 3))


    #theory of above simulation
    p2b = (3 * lam) / (mu_2 + lam)

    w_1b = 1 / (mu_1 - lam - 2 * lam_u)
    w_2b = (p2b / (1 - p2b)) * (1 / (2 * lam))
    theo_b = (w_1b / 2) + w_2b

    data_B['Theory'].append(round(theo_b, 3))
    data_B['Error'].append(round((sim_b - theo_b)*100/sim_b, 3))

In [ ]:
data_B

In [ ]:
x1 = [1.0, 1.625, 2.25, 2.875, 3.5]          #mu_1 = 5, mu_2 = 10, lam_u = 0.5
yb1 = [0.354, 0.432, 0.559, 0.797, 1.512]


x2 = [1.0, 1.875, 2.75, 3.625, 4.5]           #mu_1 = 30, mu_2 = 10, lam_u = 0.5
yb2 = [0.206, 0.258, 0.353, 0.566, 1.53]

x3 = [1.0, 1.75, 2.5, 3.25, 4.0]            #mu_1 = 10, mu_2 = 10, lam_u = 0.5
yb3 = [0.25, 0.3, 0.378, 0.514, 0.849]

# Evalution for the variation of lambda for C

In [ ]:
data_C = {'Lambda':[], 'Simulation':[], 'Theory':[], 'Error':[]}
n = 50

for i in np.linspace(1, 4.5, 5):
    lam, mu_1, mu_2, lam_u = i, 30, 10, 0.5  # request rate, service 1 rate, service 2 rate, update rate

    data_C['Lambda'].append(lam)
    ############################################################################
    #simulation C
    env = simpy.Environment()
    net = C(env, lam, mu_1, mu_2, lam_u)
    env.process(net.arrival())
    env.process(net.queue_2())
    env.process(net.queue_1())
    env.process(net.update_file())
    env.run(n*10**5)
    n = n-5

    l_1 = len(net.data['Departure(Q2,C1)'])
    l_2 = len(net.data['Departure(Q2,C2)'])
    l1 = int(l_1*0.4)   #margin
    l2 = int(l_2*0.4)   #margin

    w_1c = (sum(net.data['Departure(Q2,C1)'][l1:-l1+l_1]) - sum(net.data['Arrival(Q1,C1)'][l1:-l1+l_1])) / len(net.data['Departure(Q2,C1)'][l1:-l1+l_1])
    w_2c = (sum(net.data['Departure(Q2,C2)'][l2:-l2+l_2]) - sum(net.data['Arrival(Q2,C2)'][l2:-l2+l_2])) / len(net.data['Departure(Q2,C2)'][l2:-l2+l_2])
    sim_c = (w_1c + w_2c)/2
    data_C['Simulation'].append(round(sim_c, 5))


    #theory of above simulation
    p2c = (2 * lam + lam * (mu_1 / (mu_1 + mu_2))) / (mu_2 + lam * (mu_1 / (mu_1 + mu_2)))

    w_1c = 1 / (mu_1 - lam - 2 * lam_u)
    w_2c = (p2c / (1 - p2c)) * (1 / (2 * lam))

    theo_c = (w_1c / 2) + w_2c

    data_C['Theory'].append(round(theo_c, 3))
    data_C['Error'].append(round((sim_c - theo_c)*100/sim_c, 3))

In [ ]:
data_C

In [ ]:
x1 = [1.0, 1.625, 2.25, 2.875, 3.5]                     #mu_1 = 5, mu_2 = 10, lam_u = 0.5
yc1 = [0.30919, 0.37937, 0.49077, 0.70392, 1.35608]

x2 = [1.0, 1.875, 2.75, 3.625, 4.5]                     #mu_1 = 30, mu_2 = 10, lam_u = 0.5
yc2 = [0.19187, 0.24454, 0.33718, 0.53775, 1.43021]

x3 =  [1.0, 1.75, 2.5, 3.25, 4.0]                       #mu_1 = 10, mu_2 = 10, lam_u = 0.5
yc3 = [0.22, 0.265, 0.337, 0.46, 0.761]

# Theorical calculations

In [ ]:
Ta, Tb, Tc, lambda_ = [], [], [], []
Aa, Ab, Ac = [], [], []
mu_1, mu_2, lam_u, N = 5, 10, 0.5, 10

for lam in np.linspace(1, 3.5 , 200):
    lambda_.append(lam)

    ################################  theory case A
    p1a = (3 * lam + 6 * lam_u) / (2 * mu_1 + lam + 2 * lam_u)
    p2a = (5 * lam + lam * (mu_1)/(mu_1 + mu_2)) / (2 * mu_2 + lam + lam * (mu_1)/(mu_1 + mu_2))

    w_1a = (p1a / (1 - p1a)) * (1 / (lam + 2 * lam_u))
    w_2a = (p2a / (1 - p2a)) * (1 / (2 * lam))

    result_a = (w_1a/2 + w_2a)
    Ta.append(result_a)

    ################################  theory case B
    p2b = (3 * lam) / (mu_2 + lam)

    w_1b = 1 / (mu_1 - lam - 2 * lam_u)
    w_2b = (p2b / (1 - p2b)) * (1 / (2 * lam))

    result_b = (w_1b / 2) + w_2b
    Tb.append(result_b)

    ################################  theory case C
    p2c = (2 * lam + lam * (mu_1 / (mu_1 + mu_2))) / (mu_2 + lam * (mu_1 / (mu_1 + mu_2)))

    w_1c = 1 / (mu_1 - lam - 2 * lam_u)
    w_2c = (p2c / (1 - p2c)) * (1 / (2 * lam))

    result_c = (w_1c / 2) + w_2c
    Tc.append(result_c)


    ################################ AoI
    aa = (((1 + N)/(lam_u) + 2/mu_1 + 2/mu_2) + (2/mu_1 + w_2a))/2
    ab = (((1 + 2 * N)/(2 * lam_u) + 1/mu_1 + 1/mu_2) + (1/mu_1 + w_2b))/2
    ac = (((1 + 2 * N)/(2 * lam_u) + 1/mu_1 + 1/mu_2 )+ (1/mu_1 + w_2c))/2

    Aa.append(aa)
    Ab.append(ab)
    Ac.append(ac)

In [ ]:
import numpy as np

Ta2, Tb2, Tc2, lambda_2 = [], [], [], []
Aa2, Ab2, Ac2 = [], [], []
mu_1, mu_2, lam_u, N = 30, 10, 0.5, 10

for lam in np.linspace(1, 4.5 , 200):
    lambda_2.append(lam)

    ################################  theory case A
    p1a = (3 * lam + 6 * lam_u) / (2 * mu_1 + lam + 2 * lam_u)
    p2a = (5 * lam + lam * (mu_1)/(mu_1 + mu_2)) / (2 * mu_2 + lam + lam * (mu_1)/(mu_1 + mu_2))

    w_1a = (p1a / (1 - p1a)) * (1 / (lam + 2 * lam_u))
    w_2a = (p2a / (1 - p2a)) * (1 / (2 * lam))


    result_a = (w_1a/2 + w_2a)
    Ta2.append(result_a)

    ################################  theory case B
    p2b = (3 * lam) / (mu_2 + lam)

    w_1b = 1 / (mu_1 - lam - 2 * lam_u)
    w_2b = (p2b / (1 - p2b)) * (1 / (2 * lam))

    result_b = (w_1b / 2) + w_2b
    Tb2.append(result_b)

    ################################  theory case C
    p2c = (2 * lam + lam * (mu_1 / (mu_1 + mu_2))) / (mu_2 + lam * (mu_1 / (mu_1 + mu_2)))

    w_1c = 1 / (mu_1 - lam - 2 * lam_u)
    w_2c = (p2c / (1 - p2c)) * (1 / (2 * lam))

    result_c = (w_1c / 2) + w_2c
    Tc2.append(result_c)



    aa = (((1 + N)/(lam_u) + 2/mu_1 + 2/mu_2) + (2/mu_1 + w_2a))/2
    ab = (((1 + 2 * N)/(2 * lam_u) + 1/mu_1 + 1/mu_2) + (1/mu_1 + w_2b))/2
    ac = (((1 + 2 * N)/(2 * lam_u) + 1/mu_1 + 1/mu_2 )+ (1/mu_1 + w_2c))/2

    Aa2.append(aa)
    Ab2.append(ab)
    Ac2.append(ac)

In [ ]:
Ta3, Tb3, Tc3, lambda_3 = [], [], [], []
Aa3, Ab3, Ac3 = [], [], []
mu_1, mu_2, lam_u, N = 10, 10, 0.5, 10

for lam in np.linspace(1, 4 , 100):
    lambda_3.append(lam)

    ################################  theory case A
    p1a = (3 * lam + 6 * lam_u) / (2 * mu_1 + lam + 2 * lam_u)
    p2a = (5 * lam + lam * (mu_1)/(mu_1 + mu_2)) / (2 * mu_2 + lam + lam * (mu_1)/(mu_1 + mu_2))

    w_1a = (p1a / (1 - p1a)) * (1 / (lam + 2 * lam_u))
    w_2a = (p2a / (1 - p2a)) * (1 / (2 * lam))

    result_a = (w_1a/2 + w_2a)
    Ta3.append(result_a)

    ################################  theory case B
    p2b = (3 * lam) / (mu_2 + lam)

    w_1b = 1 / (mu_1 - lam - 2 * lam_u)
    w_2b = (p2b / (1 - p2b)) * (1 / (2 * lam))

    result_b = (w_1b / 2) + w_2b
    Tb3.append(result_b)

    ################################  theory case C
    p2c = (2 * lam + lam * (mu_1 / (mu_1 + mu_2))) / (mu_2 + lam * (mu_1 / (mu_1 + mu_2)))

    w_1c = 1 / (mu_1 - lam - 2 * lam_u)
    w_2c = (p2c / (1 - p2c)) * (1 / (2 * lam))

    result_c = (w_1c / 2) + w_2c
    Tc3.append(result_c)




    ################################ AoI
    aa = (((1 + N)/(lam_u) + 2/mu_1 + 2/mu_2) + (2/mu_1 + w_2a))/2
    ab = (((1 + 2 * N)/(2 * lam_u) + 1/mu_1 + 1/mu_2) + (1/mu_1 + w_2b))/2
    ac = (((1 + 2 * N)/(2 * lam_u) + 1/mu_1 + 1/mu_2 )+ (1/mu_1 + w_2c))/2

    Aa3.append(aa)
    Ab3.append(ab)
    Ac3.append(ac)

# Plodtting

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})


fig = plt.figure(figsize=[3.5,1.5], dpi=300)
########################################## Delay
plt.subplot(1, 2, 1)
plt.plot(lambda_, Ta, label = "WCWD-Analytical", linestyle='--', linewidth=0.5, color='m')
plt.scatter(x1, ya1, 1, label = "WCWD-Simulation", marker='<')

plt.plot(lambda_, Tb, label = "PCWD-Analytical", linewidth=0.5, color='c')
plt.scatter(x1, yb1, 1, label = "PCWD-Simulation", marker='.')

plt.plot(lambda_, Tc, label = "PCPD-Analytical", linestyle='-.', color='red', linewidth=0.5)
plt.scatter(x1, yc1, 1, label = "PCPD-Simulation",marker='x')

plt.ylabel('Average Delay')
plt.xlabel('Lambda(λ)')

plt.grid(linewidth=0.25)
plt.legend(fontsize=4)

########################################## AoI
plt.subplot(1, 2, 2)
plt.plot(lambda_, Aa, label = "WCWD-Analytical", linestyle='--',linewidth=0.5,  color='m')
plt.plot(lambda_, Ab, label = "PCWD-Analytical",linewidth=0.5, color='c')
plt.plot(lambda_, Ac, label = "PCPD-Analytical", linewidth=0.5, linestyle='-.',  color='red')

plt.ylabel('Average AoI')
plt.xlabel('Lambda(λ)')

plt.grid(linewidth=0.25)
plt.legend(fontsize=4)


SMALL_SIZE = 5
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels

plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.savefig('ieee_conference_plot_1.png', dpi=1000, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

plt.figure(figsize=[3.5,1.5], dpi=300)
########################################## Delay 2
plt.subplot(1, 2, 1)
plt.plot(lambda_2, Ta2, label = "WCWD-Analytical", linestyle='--', linewidth=0.5, color='m')
plt.scatter(x2, ya2, 1, label = "WCWD-Simulation",marker='<')

plt.plot(lambda_2, Tb2, label = "PCWD-Analytical", linewidth=0.5, color='c')
plt.scatter(x2, yb2, 1, label = "PCWD-Simulation", marker='.')

plt.plot(lambda_2, Tc2, label = "PCPD-Analytical", linestyle='-.', color='red', linewidth=0.5)
plt.scatter(x2, yc2, 1,label = "PCPD-Simulation", marker='x')

plt.ylabel('Average Delay')
plt.xlabel('Lambda(λ)')

plt.grid(linewidth=0.25)
plt.legend(fontsize=4)

########################################## AoI 2
plt.subplot(1, 2, 2)
plt.plot(lambda_2, Aa2, label = "WCWD-Analytical", linestyle='--',linewidth=0.5,  color='m')
plt.plot(lambda_2, Ab2, label = "PCWD-Analytical",linewidth=0.5, color='c')
plt.plot(lambda_2, Ac2, label = "PCPD-Analytical", linewidth=0.5, linestyle='-.',  color='red')

plt.ylabel('Average AoI')
plt.xlabel('Lambda(λ)')

plt.grid(linewidth=0.25)
plt.legend(fontsize=4)


SMALL_SIZE = 5
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels

plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.savefig('ieee_conference_plot_2.png', dpi=1000, bbox_inches='tight', transparent=True)
plt.show()

In [ ]:
from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

plt.figure(figsize=[3.5,1.5], dpi=300)
########################################## Delay 2
plt.subplot(1, 2, 1)
plt.plot(lambda_3, Ta3, label = "WCWD-Analytical", linestyle='--', linewidth=0.5, color='m')
plt.scatter(x3, ya3, 1, label = "WCWD-Simulation",marker='<')

plt.plot(lambda_3, Tb3, label = "PCWD-Analytical", linewidth=0.5, color='c')
plt.scatter(x3, yb3, 1, label = "PCWD-Simulation", marker='.')

plt.plot(lambda_3, Tc3, label = "PCPD-Analytical", linestyle='-.', color='red', linewidth=0.5)
plt.scatter(x3, yc3, 1,label = "PCPD-Simulation", marker='x')

plt.ylabel('Average Delay')
plt.xlabel('Lambda(λ)')

plt.grid(linewidth=0.25)
plt.legend(fontsize=4)

########################################## AoI 2
plt.subplot(1, 2, 2)
plt.plot(lambda_3, Aa3, label = "WCWD-Analytical", linestyle='--',linewidth=0.5,  color='m')
plt.plot(lambda_3, Ab3, label = "PCWD-Analytical",linewidth=0.5, color='c')
plt.plot(lambda_3, Ac3, label = "PCPD-Analytical", linewidth=0.5, linestyle='-.',  color='red')

plt.ylabel('Average AoI')
plt.xlabel('Lambda(λ)')

plt.grid(linewidth=0.25)
plt.legend(fontsize=4)


SMALL_SIZE = 4
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels

plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.savefig('ieee_conference_plot_3.png', dpi=1000, bbox_inches='tight', transparent=True)
plt.show()